In [142]:
import pandas as pd
df = pd.read_pickle("OPIF_04-06.pkl")
select = 'KRWF'

In [146]:
df['callsign'].value_counts()[:10]

WHNS       229
WMUR-TV    211
KRWF       180
KSAX       180
KFMB-TV    169
KTTC       159
WPTZ       156
KBJR-TV    152
KRII       152
KMPH-TV    147
Name: callsign, dtype: int64

In [147]:
dfquery = df[(df.callsign == select)]

In [20]:
import os
path = select + ' Files/'
try:
    os.mkdir(path)
except OSError:
    print ("Creation of the directory %s failed" % path)
else:
    print ("Successfully created the directory %s " % path)

Successfully created the directory KRWF Files/ 


In [21]:
import requests, requests_cache, time
from IPython.display import clear_output
requests_cache.install_cache()

In [22]:
dfquery.reset_index(drop=True, inplace=True)

In [23]:
n = 0
errorCount = 0
for url, name in zip(dfquery.file_url, dfquery.file_name):
    print("Downloading file {}/{}".format(n+1,len(dfquery)))
    
    r = requests.get(url, stream = True)
    if (r.status_code != 200): 
        print(url, r.text)
        errorCount += 1
        time.sleep(2)
    else: 
        with open(path + name + '.pdf', 'wb') as f:
            f.write(r.content)
        dfquery.loc[n,'file_path'] = path + name + '.pdf'
    if not getattr(r, 'from_cache', False):
        time.sleep(0.25)
    clear_output(wait = True)
    n += 1
print("Number of errors: ", errorCount)

Number of errors:  5


In [28]:
# import shutil
# shutil.make_archive(select + ' Files', 'zip', select + ' Files')

'C:\\Users\\Henry Weng\\PythonNotebooks\\KSAX Files.zip'

#### Scrapes all the PDFs and appends resulting strings as a dataframe column.

In [28]:
from io import StringIO
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfparser import PDFParser

def scrape(pdf):
    output_string = StringIO()
    with open(pdf, 'rb') as in_file:
        parser = PDFParser(in_file)
        doc = PDFDocument(parser)
        rsrcmgr = PDFResourceManager()
        device = TextConverter(rsrcmgr, output_string, laparams=LAParams())
        interpreter = PDFPageInterpreter(rsrcmgr, device)
        for page in PDFPage.create_pages(doc):
            interpreter.process_page(page)
    return output_string.getvalue()

In [33]:
dfquery = dfquery[dfquery['file_path'].notnull()]

In [34]:
dfquery['scrape_output'] = dfquery['file_path'].apply(scrape)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [36]:
dfquery.to_pickle('dfquery.pkl')

In [63]:
## Resuming session
import pandas as pd
dfq = pd.read_pickle('dfquery.pkl')

In [64]:
dfq.reset_index(drop=True, inplace=True)

In [3]:
dfq['scrape_output'].str.contains('Contract Date').value_counts()

True     110
False     65
Name: scrape_output, dtype: int64

In [4]:
dfq['scrape_output'].str.contains('Flight Date').value_counts()

False    110
True      65
Name: scrape_output, dtype: int64

#### We want to find the search parameter for flight/contract dates and validate that they work for each file.

In [70]:
f_dates = []
for row in dfq[dfq['scrape_output'].str.contains('Flight Date')]['scrape_output']:
    start = row.find('Flight Date') + 14
    end = start + 23
    f_dates.append(row[start:end])
f_dates[:5]

['03/03/20 - 03/03/20\n\nAd',
 '\n\n03/03/20 - 03/03/20\n\n',
 '03/02/20 - 03/03/20\n\nAd',
 '\n\n03/02/20 - 03/03/20\n\n',
 '\n\n02/26/20 - 03/03/20\n\n']

In [71]:
c_dates = []
for row in dfq[dfq['scrape_output'].str.contains('Contract Date')]['scrape_output']:
    start = row.find('Contract Date') + 14
    end = start + 50
    c_dates.append(row[start:end])
c_dates[:5]

['\n\nEstimate #\n\n03/02/20 - 03/03/20\n\n7173\n\nAdvertise',
 '\n\nEstimate #\n\n03/03/20 - 03/03/20\n\n7173\n\nAdvertise',
 '\n\nEstimate #\n\n03/02/20 - 03/03/20\n\n7173\n\nAdvertise',
 '\n\nEstimate #\n\n02/26/20 - 03/03/20\n\n69\n\nAdvertiser\n',
 '\n\nEstimate #\n\n02/26/20 - 03/03/20\n\n69\n\nAdvertiser\n']

In [72]:
import re
dateRegex = re.compile(r'((\d{2})/(\d{2})/(\d{2}) - (\d{2})/(\d{2})/(\d{2}))')
m = re.search(dateRegex, c_dates[0])
m.group()

'03/02/20 - 03/03/20'

In [73]:
n = re.search(dateRegex, f_dates[0])
n.group()

'03/03/20 - 03/03/20'

#### Now, we generalize to all rows. 

In [74]:
import numpy as np
dfq['f_start'] = np.where(dfq['scrape_output'].str.contains('Flight Date'),
                             dfq['scrape_output'].str.find('Flight Date') + 14, 0)
dfq['f_end'] = np.where(dfq['scrape_output'].str.contains('Flight Date'),
                             dfq['scrape_output'].str.find('Flight Date') + 37, 0)
dfq['c_start'] = np.where(dfq['scrape_output'].str.contains('Contract Date'),
                             dfq['scrape_output'].str.find('Contract Date') + 14, 0)
dfq['c_end'] = np.where(dfq['scrape_output'].str.contains('Contract Date'),
                             dfq['scrape_output'].str.find('Contract Date') + 50, 0)

In [75]:
## Grabs the desired section from each scrape_output.
f_dates = pd.Series([A[B:C] for A,B,C in zip(dfq.scrape_output, dfq.f_start, dfq.f_end)])
c_dates = pd.Series([A[B:C] for A,B,C in zip(dfq.scrape_output, dfq.c_start, dfq.c_end)])

In [ ]:
# flight_dates = D.replace(to_replace=r'[a-zA-Z\n]',value='',regex=True)

In [76]:
dfq['Flight_Date'] = f_dates.str.extract(dateRegex)[0]
dfq['Contract_Date'] = c_dates.str.extract(dateRegex)[0]

#### Gets the highest dollar amount.

In [77]:
def getTotal(in_string):
    amtRegex = re.compile(r'(\$)(\d{1,3})(,\d{3})?(.\d{2})?')
    mo = re.findall(amtRegex, in_string)
    numbers = []
    for match in mo:
        entry = ''
        for j in match:
            if(j != '$'): entry += j
        numbers.append(entry)
    for i in range(len(numbers)):
        numbers[i] = numbers[i].replace(',', '')
    num_array = list(map(float, numbers))
    if not num_array: 
        return 0
    return max(num_array)

In [78]:
dfq['Amount'] = dfq['scrape_output'].apply(getTotal)

In [79]:
dfq['Amount']

0        4230.00
1         465.00
2        4230.00
3         465.00
4         465.00
         ...    
170     24775.00
171     24775.00
172     23565.00
173    130394.74
174      1210.00
Name: Amount, Length: 175, dtype: float64

#### Attempts to get the total spots field.

In [130]:
def getNumberSpots(in_string):
    if(in_string.count('Total Spots') > 0):
        start = in_string.rfind('Total Spots') + 11
        end = start + 15  # originally 5
        return in_string[start:end]
    elif(in_string.count('Totals') > 0):
        start = in_string.rfind('Totals') + 6
        end = start + 15 # originally 6
        return in_string[start:end]
    return 'N/A'

In [131]:
nums = dfq['scrape_output'].apply(getNumberSpots)
nums[:10]

0    \n\n24\n\n24\n\n$4,23
1    \n\n24\n\n$465.00\n\n
2    \n\n25\n\n25\n\n$4,23
3      \n\n24\n\n8 03/03/2
4      \n\n24\n\n$465.00\n
5      \n\n25\n\nPayment T
6       \n\n25\n\n$4,230.0
7    \n\n45\n\n45\n\n$23,2
8      \n\n133\n\n$51,695.
9    \n\n97\n\n97\n\n$44,3
Name: scrape_output, dtype: object

In [135]:
numRegex = re.compile(r'(\n)(\d{1,3})(\n)(\n)(\$)')
n = re.search(numRegex, nums[1])
n.group()

'\n24\n\n$'

#### Generalizing for all rows.

In [136]:
dfq['n_start'] = np.where(dfq['scrape_output'].str.contains('Totals'),
                             dfq['scrape_output'].str.rfind('Totals') + 6, 0)
dfq['n_end'] = np.where(dfq['scrape_output'].str.contains('Totals'),
                             dfq['scrape_output'].str.rfind('Totals') + 21, 0)
dfq['t_start'] = np.where(dfq['scrape_output'].str.contains('Total Spots'),
                             dfq['scrape_output'].str.rfind('Total Spots') + 11, 0)
dfq['t_end'] = np.where(dfq['scrape_output'].str.contains('Total Spots'),
                             dfq['scrape_output'].str.rfind('Total Spots') + 26, 0)

In [137]:
total_spots = pd.Series([A[B:C] for A,B,C in zip(dfq.scrape_output, dfq.t_start, dfq.t_end)])
totals = pd.Series([A[B:C] for A,B,C in zip(dfq.scrape_output, dfq.n_start, dfq.n_end)])

In [138]:
dfq['Total_Spots'] = total_spots.str.extract(numRegex)[1]
dfq['Totals'] = totals.str.extract(numRegex)[1]

In [140]:
dfq['Total_Spots']

0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
      ... 
170    NaN
171    NaN
172    NaN
173    NaN
174    NaN
Name: Total_Spots, Length: 175, dtype: object

In [143]:
## Export to Excel
dfexp = dfq[['candidate','date','callsign','file_name',
             'Flight_Date','Contract_Date','Amount','Totals','Total_Spots','file_url']]
dfexp.to_excel(select + '-04-06.xlsx')